## Installing packages
* pip install lightgbm
* conda install -c conda-forge xgboost
* pip install xgboost

## **Stack OverFlow: EDA & ML**  

**Elijah Zolduoarrati**  
**Approaches and Techniques:**

* EDA with Pandas and Seaborn
* Find features with strong correlation to target variables questions and answers
* Data preprocessing, converting categorical features mainly (country) to numerical
* apply the basic Regression models of sklearn 
* use gridsearchCV to find the best parameters for each model
* compare the performance of the Regressors and choose best one

**The notebook is organized as follows:**

* **[Part 0: Imports, Settings and switches, Global functions](#Part-0-:-Imports,-Settings,-Functions)**  
* import libraries  
* settings for number of cross validations  
* define functions that are used often

* **[Part 1: Exploratory Data Analysis](#Part-1:-Exploratory-Data-Analysis)**  
1.1 Get an overview of the features (numerical and categorical) and first look on the target variables questions and answers
[shape, info, head and describe](#shape,-info,-head-and-describe)  
[Distribution of the target variable Q](#The-target-variable-:-Distribution-of-questions-and-answers)  
[Numerical and Categorical features](#Numerical-and-Categorical-features)  
[List of features with missing values](#List-of-features-with-missing-values) and Filling missing values using [log transform](#log-transform)  
1.2 Relation of all features to target questions and answers  
[Seaborn regression plots for numerical features](#Plots-of-relation-to-target-for-all-numerical-features)  
[List of numerical features and their correlation coefficient to target](#List-of-numerical-features-and-their-correlation-coefficient-to-target)  
[Seaborn boxplots for categorical features](#Relation-to-questions-and-answers-for-all-categorical-features)  
[List of categorical features and their unique values](#List-of-categorical-features-and-their-unique-values)  
1.3 Determine the columns that show strong correlation to target  
[Correlation matrix 1](#Correlation-matrix-1) : all numerical features determine features with largest correlation to questions and answers

* **[Part 2: Data wrangling](#Part-2:-Data-wrangling)**  
[Dropping all columns with weak correlation to questions and answers](#Dropping-all-columns-with-weak-correlation-to-questions-and-answers)  
[Convert categorical columns to numerical](#Convert-categorical-columns-to-numerical)  
[Checking correlation to SalePrice for the new numerical columns](#Checking-correlation-to-questions-and-answers-for-the-new-numerical-columns)  
use only features with strong correlation to target  
[Correlation Matrix 2 (including converted categorical columns)](#Correlation-Matrix-2-:-All-features-with-strong-correlation-to-questions-and-answers)  
Create datasets for ML algorithms:                                                                          
[OneHotEncoder](#OneHotEncoder)  
[StandardScaler](#StandardScaler)

* **[Part 3: Scikit-learn basic regression models and comparison of results](#Part-3:-Scikit-learn-basic-regression-models-and-comparison-of-results)**  
implement GridsearchCV with RMSE metric for Hyperparameter tuning for these models from sklearn:  
[Linear Regression](#Linear-Regression)  
[Ridge](#Ridge)  
[Lasso](#Lasso)  
[Elastic Net](#Elastic-Net)  
[Stochastic Gradient Descent](#SGDRegressor)  
[DecisionTreeRegressor](#DecisionTreeRegressor)  
[Random Forest Regressor](#RandomForestRegressor)  
[KNN Regressor](#KNN-Regressor)  
Baed on RMSE metric, compare performance of the regressors with their optimized parameters, then explore correlation of the predictions and make submission with mean of best models plot comparison:             
[RMSE of all models](#Comparison-plot:-RMSE-of-all-models)  
[Correlation of model results](#Correlation-of-model-results)  
Mean of best models


Note on scores:  
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed questions and answers. (Taking logs means that errors in predicting questions and answers will affect the result equally.)

# Part 0 : Imports, Settings, Functions

In [4]:
#Visualizing Lib
import seaborn as sns
import matplotlib.pyplot as plt

#Math Lib for some statistics
from scipy import stats
%matplotlib inline
sns.set()

# df preprocessing Lib
import pandas as pd
import numpy as np
pd.set_option('max_columns', 105)

# AI preprocessing lib
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
#import sklearn.impute.SimpleImputer
from sklearn.model_selection import train_test_split 

# ML Lib
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
import sklearn.linear_model as linear_model
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor

# warning supressor
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore")
#importing necessary models and libraries

### Settings and switches

* Here we can choose settings for optimal performance and runtime. 
* For example, nr_cv sets the number of cross validations used in GridsearchCV, and min_val_corr is the minimum value for the correlation coefficient to the target (only features with larger correlation will be used). 

In [5]:
# setting the number of cross validations used in the Model part 
nr_cv = 5

# switch for using log values for SalePrice and features     
use_logvals = 1    
# target used for correlation 
target_1 = 'Questions_log'
target_2 = 'Answers_log'    
# only columns with correlation above this threshold value  
# are used for the ML Regressors in Part 3
min_val_corr = 0.4    
    
# switch for dropping columns that are similar to others already used and show a high correlation to these     
drop_similar = 1

## Initiate functions:

In [6]:
def get_best_score(grid):
    
    best_score = np.sqrt(-grid.best_score_)
    print(best_score)    
    print(grid.best_params_)
    print(grid.best_estimator_)
    return best_score

def print_cols_large_corr(df, nr_c, targ) :
    corr = df.corr()
    corr_abs = corr.abs()
    print (corr_abs.nlargest(nr_c, targ)[targ])

def plot_corr_matrix(df, nr_c, targ) :
    
    corr = df.corr()
    corr_abs = corr.abs()
    cols = corr_abs.nlargest(nr_c, targ)[targ].index
    cm = np.corrcoef(df[cols].values.T)

    plt.figure(figsize=(nr_c/1.5, nr_c/1.5))
    sns.set(font_scale=1.25)
    sns.heatmap(cm, linewidths=1.5, annot=True, square=True, 
                fmt='.2f', annot_kws={'size': 10}, 
                yticklabels=cols.values, xticklabels=cols.values
               )
    plt.show()

In [7]:
from subprocess import check_output, call 
print(check_output(["dir", "C:\\Data\\all.csv"],shell=True).decode("utf8"))

 Volume in drive C is Windows
 Volume Serial Number is 5843-B50F

 Directory of C:\Data

13/12/2019  11:06 AM        11,621,782 all.csv
               1 File(s)     11,621,782 bytes
               0 Dir(s)  357,122,838,528 bytes free



In [8]:
#Importing data into dataframe
df =  pd.read_csv('C://Data/all.csv')

In [9]:
#Displaying dataframe
df.head()

,Id,country,AboutMe_length,activity_in_months,UpVotes,DownVotes,Reputation,Views,badges,Q_comments,A_comments,P_questions,P_answers,p_ModeratorNomination,p_PrivilegeWiki,p_TagWiki,p_TagWikiExerpt,p_Wiki,p_WikiPlaceholder,ph_InitialTitle,ph_EditTitle,ph_InitialBody,ph_InitialTags,ph_EditBody,ph_EditTags,ph_RollbackTitle,ph_RollbackBody,ph_PostReopened,ph_RollbackTags,ph_PostClosed,ph_PostDeleted,ph_PostUndeleted,ph_QuestionMerged,ph_PostMigrated,ph_CommunityOwned,ph_PostLocked,ph_PostUnlocked,ph_QuestionUnprotected,ph_QuestionUnmerged,ph_PostDisassociated,ph_QuestionProtected,ph_SuggestedEditApplied,ph_UnknownDevRelatedEvent,ph_PostTweeted,ph_VoteNullificationByDev,ph_UnknownSuggestionEvent,ph_PostUnmigrated,ph_UnknownModeratorEvent,ph_UnknownEvent,ph_CommentDiscussionMovedToChat,ph_PostNoticeRemoved,ph_PostNoticeAdded,ph_PostMergeSource,ph_PostMigratedAway,ph_PostMergeDestination,ph_PostMigratedHere
0,7784035,usa,1407.0,0.0,0,0,1,2,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9126518,usa,622.0,0.0,0,0,1,0,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10021482,usa,370.0,0.0,0,0,1,1,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2739419,usa,400.0,0.0,0,0,1,1,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1440631,usa,32.0,0.0,0,0,1,0,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print("-"*60)
print("List features with missing values and their percentage")
print("-"*60)
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

------------------------------------------------------------
List features with missing values and their percentage
------------------------------------------------------------


,Total,Percent
ph_PostMigratedHere,140184,1.000000
ph_UnknownSuggestionEvent,140184,1.000000
ph_QuestionMerged,140184,1.000000
ph_PostMigrated,140184,1.000000
p_Wiki,140184,1.000000
ph_QuestionUnmerged,140184,1.000000
ph_PostDisassociated,140184,1.000000
ph_SuggestedEditApplied,140184,1.000000
ph_UnknownDevRelatedEvent,140184,1.000000
ph_VoteNullificationByDev,140184,1.000000


In [14]:
print("Completely missing features (1.000000) will be eliminated as including them in the analysis will be useless as the rest of features will remain for now")

Completely missing features (1.000000) will be eliminated as including them in the analysis will be useless as the rest of features will remain for now


In [16]:
# subtitute missing values by zero
#df = df.fillna(0)
## Displaying modified dataframe
#df.head()
## creating a variable vector
# df = df.iloc[:,[0,6,7,8,9,10,11,12,13,14,15,16,17]]
## Or
# vec = df.loc[:,['Id','DisplayName','Location','country','AboutMe_length','activity_in_months','UpVotes','DownVotes','Reputation','Views','badges','Q_comments','A_comments','P_questions','P_answers']]
df.drop(['ph_PostMigratedHere','ph_UnknownSuggestionEvent','ph_QuestionMerged','ph_PostMigrated','ph_PostMigrated','p_Wiki','ph_QuestionUnmerged','ph_PostDisassociated','ph_SuggestedEditApplied','ph_UnknownDevRelatedEvent','ph_VoteNullificationByDev','ph_PostTweeted','ph_PostUnmigrated','ph_UnknownModeratorEvent','ph_UnknownEvent','ph_CommentDiscussionMovedToChat','p_PrivilegeWiki','p_WikiPlaceholder'], axis= 1, inplace=True);

In [17]:
df.head()

,Id,country,AboutMe_length,activity_in_months,UpVotes,DownVotes,Reputation,Views,badges,Q_comments,A_comments,P_questions,P_answers,p_ModeratorNomination,p_TagWiki,p_TagWikiExerpt,ph_InitialTitle,ph_EditTitle,ph_InitialBody,ph_InitialTags,ph_EditBody,ph_EditTags,ph_RollbackTitle,ph_RollbackBody,ph_PostReopened,ph_RollbackTags,ph_PostClosed,ph_PostDeleted,ph_PostUndeleted,ph_CommunityOwned,ph_PostLocked,ph_PostUnlocked,ph_QuestionUnprotected,ph_QuestionProtected,ph_PostNoticeRemoved,ph_PostNoticeAdded,ph_PostMergeSource,ph_PostMigratedAway,ph_PostMergeDestination
0,7784035,usa,1407.0,0.0,0,0,1,2,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9126518,usa,622.0,0.0,0,0,1,0,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10021482,usa,370.0,0.0,0,0,1,1,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2739419,usa,400.0,0.0,0,0,1,1,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1440631,usa,32.0,0.0,0,0,1,0,1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Create training and testing sets
df_train,df_test= train_test_split(df, test_size = 0.2, random_state = 0)

# Part 1: Exploratory Data Analysis

## 1.1 Overview of features and relation to target

Let's get a first overview of the train and test dataset
* How many rows and columns are there?  
* What are the names of the features (columns)?  
* Which features are numerical, which are categorical?  
* How many values are missing?  
The **shape** and **info** methods answer these questions. Whereas, the **head** displays some rows of the dataset **describe** gives a summary of the statistics (only for numerical columns)

### Shape, Info, Head & Describe -----> Functions

In [ ]:
print('-'*100)
print('training sample size')
print(df_train.shape)
print('-'*100)
print('testing sample size')
print(df_test.shape)
print('-'*100)
print('training sample features description')
print(df_train.info())
print('-'*100)
print('testing sample features description')
print(df_train.info())
print('-'*100)

* It seems like the trainning and tesing dataframe *(df)* vector *(vec)* consists of 13 columns (12 features excluding Id), as for the training df vec, it has 112147 entries (number of rows). On the other hand,  df test vec has 28037 entries.  
* There are lots of info that is probably related to the dependent variables (target) questions and answers such as badges, reputaion, etc...   
* Maybe other features are not so important for predicting the target, also there might be a strong correlation for some of the features (like activity_in_month).
* There are missing in some columns and it seems some countries tend to have more missing data than others, we are going to deal with missing data accordingly in a later stage

In [ ]:
# displaying a sample from the training dataframe
df_train.head()

In [ ]:
# displaying a descriptive stats regarding the training dataframe 
df_train.describe()

In [ ]:
# displaying a sample from the testing dataframe
df_test.head()

In [ ]:
# displaying a descriptive stats regarding the testing dataframe
df_test.describe()

## Distribution of target variables (Questions and Answers)

In [ ]:
## optimising plots size
# data = np.random.normal(0, 1, 3)
# array([-1.18878589,  0.59627021,  1.59895721])
# ploty = plt.figure(figsize=(20, 15))
# sns.boxplot(x=data);

In [ ]:
# ~ conversion error prevention **
# df_train = df_train.fillna(0)
# Seaborn 0_0
print('-'*100)
print('-'*100)
plt.figure(figsize=(20, 15))
sns.distplot(df_train['P_questions'].dropna());
# skewness and Kurtosis
print("Skewness: %f" % df_train['P_questions'].skew());
print("Kurtosis: %f" % df_train['P_questions'].kurt());
print('-'*100)
print('-'*100)
# ValueError: cannot convert float NaN to integer --- Error --- convertion is required ---> utilising fillna in early phase

In [ ]:
print('-'*100)
print('-'*100)
plt.figure(figsize=(20, 15))
sns.distplot(df_train['P_answers'].dropna());
# skewness and Kurtosis
print("Skewness: %f" % df_train['P_answers'].skew());
print("Kurtosis: %f" % df_train['P_answers'].kurt());
print('-'*100)
print('-'*100)

* As we can see, the target variable for both questions and answers is not normally distributed. 
* This behaviour can leads to performance reduction in the ML regression modeling due the fact that some models assume normal distribution.
* Therfore a log transformation is required(see sklearn info on preprocessing) to enhance distribution visualisation.
* Before running log transformation, we need to deal with missing data in questions and answers otherwise will face the error due to zero division which will produce -1nf (minus infinity)

In [ ]:
np.seterr(divide = 'ignore')
df_train['Questions_log'] = np.where(df_train.loc[:,['P_questions']]>0, np.log(df_train.loc[:,['P_questions']]), 0)
df_train['Answers_log'] = np.where(df_train.loc[:,['P_answers']]>0, np.log(df_train.loc[:,['P_answers']]), 0)


# # alternative implementation to avoids warnings.
# loc = np.where(myarray>0)
# result2 = np.zeros_like(myarray, dtype=float)
# result2[loc] =np.log(myarray[loc])

# # answer
# myarray= np.random.randint(10,size=10)
# result = np.where(myarray>0, np.log(myarray), 0)

# # check it is giving right solution:
# print(np.allclose(result, result2))

# append columns
# df_train = pd.concat([df_train, Questions_log], axis=1)

In [ ]:
df_train.head()

In [ ]:
sns.distplot(df_train['Questions_log'].dropna());
print('-'*100)

In [ ]:
sns.distplot(df_train['Answers_log'].dropna());
print('-'*100)

In [ ]:
print("-"*100)
print("Skewness and kurtosis for questions log")
print("Skewness: %f" % df_train['Questions_log'].skew());
print("Kurtosis: %f" % df_train['Questions_log'].kurt());
print("-"*100)

In [ ]:
print("-"*100)
print("Skewness and kurtosis for answers log")
print("Skewness: %f" % df_train['Answers_log'].skew());
print("Kurtosis: %f" % df_train['Questions_log'].kurt());
print("-"*100)

In [ ]:
print("Dropping old features from the loaded dataframe")
df_train.drop("P_questions",axis=1,inplace=True);
df_train.drop("P_answers",axis=1,inplace=True);

In [ ]:
df_train.head()

In [ ]:
print("-"*100)
print("Counting numerical and categorical features exist in the dataframe")
print("-"*100)
numerical_feats = df_train.dtypes[df_train.dtypes != "object"].index
print("Number of numerical features: ", len(numerical_feats));
categorical_feats = df_train.dtypes[df_train.dtypes == "object"].index
print("Number of categorical features: ", len(categorical_feats));
print("-"*100)

In [ ]:
print("-"*35)
print("Displaying numerical features")
print("-"*35)
print(df_train[numerical_feats].columns)
print("-"*35)
print("Displaying categorical features")
print("-"*35)
print(df_train[categorical_feats].columns)

In [ ]:
print("-"*40)
print("Displaying numerical features dataframe")
print("-"*40)
df_train[numerical_feats].head()

In [ ]:
print("-"*40)
print("Displaying categorical features")
print("-"*40)
df_train[categorical_feats].head()

In [ ]:
print("-"*40)
print("List features with missing values and their percentage")
print("-"*40)
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

## Filling missing values
* we have a few columns with a large percentage of NaN entries.

In [ ]:
cols_fillna = ['A_comments','Q_comments','AboutMe_length']
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df_train[col].fillna(0,inplace=True)
    df_test[col].fillna(0,inplace=True)

In [ ]:
print("-"*40)
print("List features after handeling missing values")
print("-"*40)
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

In [ ]:
print("-"*40)
print("Missing values in train data ?")
print("-"*40)
df_train.isnull().sum().sum()

In [ ]:
print("-"*40)
print("Missing values in test data ?")
print("-"*40)
df_train.isnull().sum().sum()

## log transform

* Like the target features, other features are not normally distributed therefore it is better to tranform those features using log in df_train and df_test. 

In [ ]:
print("-"*40)
print("Checking for skewness and kurtosis:")
print("-"*40)

for col in numerical_feats:
    print('{:15}'.format(col),'    ', 
          'Skewness: {:05.2f}'.format(df_train[col].skew()) , 
          '   ' ,
          'Kurtosis: {:06.2f}'.format(df_train[col].kurt())  
         )

In [ ]:
list(set(df_train.dtypes.tolist()))

In [ ]:
df_num = df_train.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

## 1.2 Relation of features to target (SalePrice_log)
### Plots of relation to target for all numerical features

In [ ]:
nr_rows = 5
nr_cols = 2

fig, axs = plt.subplots(nr_rows, nr_cols, figsize=(nr_cols*3.5,nr_rows*3))

li_num_feats = list(numerical_feats)
li_not_plot = ['Id', 'Questions_log']
li_plot_num_feats = [c for c in list(numerical_feats) if c not in li_not_plot]


for r in range(0,nr_rows):
    for c in range(0,nr_cols):  
        i = r*nr_cols+c
        if i < len(li_plot_num_feats):
            sns.regplot(df_train[li_plot_num_feats[i]], df_train[target_1], ax = axs[r][c])
            stp = stats.pearsonr(df_train[li_plot_num_feats[i]], df_train[target_1])
            #axs[r][c].text(0.4,0.9,"title",fontsize=7)
            str_title = "r = " + "{0:.2f}".format(stp[0]) + "      " "p = " + "{0:.2f}".format(stp[1])
            axs[r][c].set_title(str_title,fontsize=11)
            
plt.tight_layout()    
plt.show()   

In [ ]:
nr_rows = 5
nr_cols = 2

fig, axs = plt.subplots(nr_rows, nr_cols, figsize=(nr_cols*3.5,nr_rows*3))

li_num_feats = list(numerical_feats)
li_not_plot = ['Id', 'Answers_log']
li_plot_num_feats = [c for c in list(numerical_feats) if c not in li_not_plot]


for r in range(0,nr_rows):
    for c in range(0,nr_cols):  
        i = r*nr_cols+c
        if i < len(li_plot_num_feats):
            sns.regplot(df_train[li_plot_num_feats[i]], df_train[target_2], ax = axs[r][c])
            stp = stats.pearsonr(df_train[li_plot_num_feats[i]], df_train[target_2])
            #axs[r][c].text(0.4,0.9,"title",fontsize=7)
            str_title = "r = " + "{0:.2f}".format(stp[0]) + "      " "p = " + "{0:.2f}".format(stp[1])
            axs[r][c].set_title(str_title,fontsize=11)
            
plt.tight_layout()    
plt.show()  

### List of numerical features and their correlation coefficient to target

**Find columns with strong correlation to target**  
Only those with r > min_val_corr are used in the ML Regressors in Part 3  
The value for min_val_corr can be chosen in global settings

In [ ]:
corr = df_train.corr()
corr_abs = corr.abs()

nr_num_cols = len(numerical_feats)
ser_corr = corr_abs.nlargest(nr_num_cols, target_1)[target_1]

cols_abv_corr_limit = list(ser_corr[ser_corr.values > min_val_corr].index)
cols_bel_corr_limit = list(ser_corr[ser_corr.values <= min_val_corr].index)

In [ ]:
print(ser_corr)
print("-"*100)
print("List of numerical features with r above min_val_corr :")
print(cols_abv_corr_limit)
print("-"*100)
print("List of numerical features with r below min_val_corr :")
print(cols_bel_corr_limit)

In [ ]:
corr = df_train.corr()
corr_abs = corr.abs()

nr_num_cols = len(numerical_feats)
ser_corr = corr_abs.nlargest(nr_num_cols, target_2)[target_2]

cols_abv_corr_limit = list(ser_corr[ser_corr.values > min_val_corr].index)
cols_bel_corr_limit = list(ser_corr[ser_corr.values <= min_val_corr].index)

In [ ]:
print(ser_corr)
print("-"*100)
print("List of numerical features with r above min_val_corr :")
print(cols_abv_corr_limit)
print("-"*100)
print("List of numerical features with r below min_val_corr :")
print(cols_bel_corr_limit)

### List of categorical features and their unique values

In [ ]:
print('-'*100)
for catg in list(categorical_feats) :
    print(df_train[catg].value_counts())
    print('-'*100)

### Relation to SalePrice for all categorical features

In [ ]:
li_cat_feats = list(categorical_feats)
nr_rows = 2
nr_cols = 2
fig, axs = plt.subplots(nr_rows, nr_cols, figsize=(nr_cols*5,nr_rows*4))

for r in range(0,nr_rows):
    for c in range(0,nr_cols):  
        i = r*nr_cols+c
        if i < len(li_cat_feats):
            sns.boxplot(x=li_cat_feats[i], y=target_1, data=df_train, ax = axs[r][c])
plt.tight_layout()    
plt.show()  

In [ ]:
li_cat_feats = list(categorical_feats)
nr_rows = 2
nr_cols = 2
fig, axs = plt.subplots(nr_rows, nr_cols, figsize=(nr_cols*5,nr_rows*4))

for r in range(0,nr_rows):
    for c in range(0,nr_cols):  
        i = r*nr_cols+c
        if i < len(li_cat_feats):
            sns.boxplot(x=li_cat_feats[i], y=target_2, data=df_train, ax = axs[r][c])
plt.tight_layout()    
plt.show()

### Correlation matrix 1
**Features with largest correlation to Questions_Log**  
all numerical features with correlation coefficient above threshold 

In [ ]:
nr_feats = len(cols_abv_corr_limit)
plot_corr_matrix(df_train, nr_feats, target_1)

In [ ]:
nr_feats = len(cols_abv_corr_limit)
plot_corr_matrix(df_train, nr_feats, target_2)

* Of those features with the largest correlation to questions and answers, some also are correlated strongly to each other.
* To avoid failures of the ML regression models due to multicollinearity, these are dropped in part 2.
* This is optional and controlled by the switch drop_similar (global settings)

# Part 2: Data wrangling

* Drop all columns with only small correlation to P_questions and P_answers
* Transform Categorical to numerical
* Handling columns with missing data
* Log values
* Drop all columns with strong correlation to similar features

In [ ]:
# # create the independent variable vector
# x = df.iloc[:,6:15].values
# # Question dependent variable vector 
# y = df.iloc[:, -2:-1].values
# # Answer dependent variable vector 
# z = df.iloc[:,-1:].values
# # create labelEncoder object to transform categorical values into integers
# x[:, 0] = LabelEncoder().fit_transform(x[:, 0])
# y = LabelEncoder().fit_transform(y)
# z = LabelEncoder().fit_transform(z)
# # creating OneHotEncoder object to transform integer categorical values into dummy categorical
# x = OneHotEncoder(categorical_features=[0]).fit_transform(x).toarray()
# # Create training and testing sets
# x_train,x_test,y_train,y_test,z_train,z_test = train_test_split(x,y,z, test_size = 0.2, random_state = 0)
# # feature scaling
# sc_x = StandardScaler()
# x_train = sc_x.fit_transform(x_train)
# x_test = sc_x.transform(x_test)